In [1]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request, jsonify


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import numpy as np
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mosina.S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mosina.S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mosina.S\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
from tensorflow.keras.models import load_model
model=load_model(r"D:\chatbot\chatbot_model.h5")
import json
import random
intents=json.loads(open(r"D:\chatbot\intents.json", encoding='utf8').read())
words=pickle.load(open(r"D:\chatbot\words.pkl","rb"))
classes=pickle.load(open(r"D:\chatbot\classes.pkl","rb"))


ModuleNotFoundError: No module named 'distutils'

In [5]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence_words",sentence_words)
    return sentence_words

In [6]:
def bow(sentence, words, show_details=True):
    # tokenize
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)



In [7]:
def predict_class(sentence, model):
    p=bow(sentence, words, show_details=False)
    print("p =",p)
    res=model.predict(np.array([p]))[0]
    print("res=",res)
    ERROR_THRESHOLD=0.25
    results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    print("result=",results)
    #sort by strength
    results.sort(key=lambda x:x[1], reverse=True)
    result_list=[]
    for r in results:
        result_list.append({"intent":classes[r[0]], "probability": str(r[1])})
    return result_list

In [8]:
def getResponse(ints, intents):
    # Initialize result variable
    result = "I'm sorry, I didn't understand that."  # Default response if no match found

    # Check if there are any intents recognized
    if ints and len(ints) > 0:
        tag = ints[0]['intent']  # Get the top intent
        for i in intents['intents']:
            if i['tag'] == tag:
                result = random.choice(i['responses'])  # Get a random response
                break

    return result





In [9]:
def chatbot_response(msg):
    ints=predict_class(msg,model)
    print("ints=",ints)
    res=getResponse(ints, intents)
    print("chatbot response=", res)
    return res

In [10]:

!pip install flask-ngrok
!pip install pyngrok
!pip install flask-cors
from flask import Flask
from flask_cors import CORS






Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
from flask import Flask, jsonify, request, Response
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
  # Only call this once


In [12]:


# Allow specific origins or allow all
app = Flask(__name__)
run_with_ngrok(app)  # Start Ngrok automatically
CORS(app)
CORS(app, resources={r"/query/*": {"origins": "*"}})


In [13]:
from flask import Response, json

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    response_data = {"response": chatbot_response(user_message)}
    return Response(json.dumps(response_data), content_type='application/json')

In [14]:
import logging
logging.basicConfig(level=logging.DEBUG)
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
run_with_ngrok(app)  # This will start Ngrok automatically
CORS(app)

In [15]:
@app.route("/", methods=['GET'])
def index():
    return jsonify({"health": "server is running successfully"})

def decrypt(msg):
    string=msg
    new_string=string.replace('+', " ")
    return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    logging.debug(f"Received query: {sentence}")
    dec_msg = decrypt(sentence)
    logging.debug(f"Decrypted query: {dec_msg}")
    response = chatbot_response(dec_msg)
    logging.debug(f"Response from chatbot: {response}")
    return jsonify({"top": {"res": response}})

if __name__=='__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 23:03:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 23:03:59] "GET / HTTP/1.1" 200 -
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4040
INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 23:04:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 23:04:00] "GET /favicon.ico HTTP/1.1" 404 -
Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\Mosina.S\AppData\Roaming\Python\Python312\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Mosina.S\AppData\Roaming\Python\Python312\site-packages\urllib3\util\connection.py", line 85, in create_connection


In [69]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import numpy as np
import pickle
from tensorflow.keras.models import load_model
model=load_model(r"D:\chatbot\chatbot_model.h5")
import json
import random
intents=json.loads(open(r"D:\chatbot\intents.json", encoding='utf8').read())
words=pickle.load(open(r"D:\chatbot\words.pkl","rb"))
classes=pickle.load(open(r"D:\chatbot\classes.pkl","rb"))
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence_words",sentence_words)
    return sentence_words
def bow(sentence,words,show_details=True):
    #tokenize
    sentence_words=clean_up_sentence(sentence)
    #bow
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if(w==s):
                #assign 1 if current word is in vocabulary position
                bag[i]=1
                if show_details:
                    print("found in bag: %s" %w)
def predict_class(sentence, model):
    p=bow(sentence, words, show_details=False)
    print("p =",p)
    res=model.predict(np.array([p]))[0]
    print("res=",res)
    ERROR_THRESHOLD=0.25
    results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    print("result=",results)
    #sort by strength
    results.sort(key=lambda x:x[1], reverse=True)
    result_list=[]
    for r in results:
        result_list.append({"intent":classes[r[0]], "probability": str(r[1])})
    return result_list    return np.array(bag)  
def getResponse(ints, intents):
    # Initialize result variable
    result = "I'm sorry, I didn't understand that."  # Default response if no match found

    # Check if there are any intents recognized
    if ints and len(ints) > 0:
        tag = ints[0]['intent']  # Get the top intent
        for i in intents['intents']:
            if i['tag'] == tag:
                result = random.choice(i['responses'])  # Get a random response
                break

    return result
def chatbot_response(msg):
    ints=predict_class(msg,model)
    print("ints=",ints)
    res=getResponse(ints, intents)
    print("chatbot response=", res)
    return res 

!pip install flask-ngrok
!pip install pyngrok
!pip install flask-cors
from flask import Flask
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # This will allow your React app to make requests to Flask
import logging
logging.basicConfig(level=logging.DEBUG)

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    logging.debug(f"Received query: {sentence}")
    dec_msg = decrypt(sentence)
    logging.debug(f"Decrypted query: {dec_msg}")
    response = chatbot_response(dec_msg)
    logging.debug(f"Response from chatbot: {response}")
    return jsonify({"top": {"res": response}}) 
from flask_cors import CORS

# Allow specific origins or allow all
CORS(app, resources={r"/query/*": {"origins": "*"}}) 
from flask import Response, json

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    response_data = {"response": chatbot_response(user_message)}
    return Response(json.dumps(response_data), content_type='application/json')
import logging
logging.basicConfig(level=logging.DEBUG)
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
run_with_ngrok(app)  # This will start Ngrok automatically
CORS(app)

@app.route("/", methods=['GET'])
def index():
    return jsonify({"health": "server is running successfully"})

def decrypt(msg):
    string=msg
    new_string=string.replace('+', " ")
    return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    logging.debug(f"Received query: {sentence}")
    dec_msg = decrypt(sentence)
    logging.debug(f"Decrypted query: {dec_msg}")
    response = chatbot_response(dec_msg)
    logging.debug(f"Response from chatbot: {response}")
    return jsonify({"top": {"res": response}})

if __name__=='__main__':
    app.run()


SyntaxError: invalid syntax (2655040766.py, line 49)

In [37]:
from flask import Response, json

@app.route('/chat', methods=['GET'])
def chat():
    user_message = request.json.get('message')
    logging.debug(f"User message: {user_message}")
    if not user_message:
        return Response(json.dumps({"error": "No message provided"}), status=400, content_type='application/json')

    response_data = {"response": chatbot_response(user_message)}
    return Response(json.dumps(response_data), content_type='application/json')


In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
run_with_ngrok(app)  # This will start Ngrok automatically
CORS(app)

@app.route("/", methods=['GET'])
def index():
    return jsonify({"health": "server is running successfully"})

def decrypt(msg):
    string=msg
    new_string=string.replace('+', " ")
    return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    logging.debug(f"Received query: {sentence}")
    dec_msg = decrypt(sentence)
    logging.debug(f"Decrypted query: {dec_msg}")
    response = chatbot_response(dec_msg)
    logging.debug(f"Response from chatbot: {response}")
    return jsonify({"top": {"res": response}})

if __name__=='__main__':
    app.run()
